# My Diet with ChatGPT
### notebook version
Designing my diet with ChatGPT

raul.arrabales | Aug. '23


## Install required packages

In [1]:
# Already satisfied in my container
# ! pip install python-dotenv
# ! pip install tiktoken
# ! pip install openai
# ! pip install panel

## Connect to my data folder

In [5]:
# Using my local container data. 
# Use the following to use GDrive: 

# Mount my drive folder
# Only for Google Colab
# from google.colab import drive
# drive.mount('/content/drive')

## Libs

In [217]:
import os
import openai
import tiktoken
from dotenv import load_dotenv, find_dotenv
import json
from pprint import pprint

## Setup

In [8]:
# _ = load_dotenv('/content/drive/My Drive/Colab Notebooks/.env') # read drive colab folder .env file
_ = load_dotenv('myenv.txt')

openai.api_key  = os.environ['OPENAI_API_KEY']

## OpenAI API Helper functions

In [9]:
''' *** get_completion(prompt, model, temp) ***
        -----------------------------------
Helper function to get a GPT 3.5 Turbo completion
Using the chat completion API
(https://platform.openai.com/docs/guides/gpt/chat-completions-api)

INPUT:
- prompt: user's prompt
- model: OpenAI model (GPT 3.5 Turbo by default)
- temp: model's temperature (0 by default)

OUTPUT:
- The instruct LLM response
'''
def get_completion(prompt, model="gpt-3.5-turbo", temp=0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp,
    )
    return response.choices[0].message["content"]

In [10]:
''' *** get_completion_from_messages(messages, model, temp) ***
        ---------------------------------------------------
Helper function to get a GPT 3.5 Turbo completion.
In this case, from the list of past messages.
Using the chat completion API
(https://platform.openai.com/docs/guides/gpt/chat-completions-api)

INPUT:
- messages: list of messages
- model: OpenAI model (GPT 3.5 Turbo by default)
- temp: model's temperature (0 by default)

OUTPUT:
- The instruct LLM response
'''
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temp=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp,
    )
    return response.choices[0].message["content"]

In [112]:
''' *** chat_with_context(prompt) ***
        -----------------------------
Gets completion using context global variable as the list of messages. 
Updates the context with current messages. 

INPUT:
- prompt: user's prompt

OUTPUT:
- Last LLM's response
'''
def chat_with_context(prompt):
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    return response

In [219]:
# From https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb 
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

# Prompt Engineering

## First conversation

In [176]:
# Starting with an empty context
context = []

### Getting the list of food categories

In [177]:
system_role_prompt = f"""
You are a nutrition expert agent with predilection for the mediterranean diet \
and extensive knowledge about healthy nutritional habits. You are also an expert in gluten-free diets.
Your mission is to answer questions about nutrition and cooking, focusing on the reducing the risk of \
metabolic syndrome.
"""

In [178]:
# List of messages, starting with system's role
context.append({'role':'system', 'content':system_role_prompt})

In [179]:
Food_prompt_0 = f"""
What are all the food categories to be considered when designing a complete diet for a male adult?
Your output should be only the list of food categories, nothing else. 
"""

In [180]:
Food_categories = chat_with_context(Food_prompt_0)

In [181]:
Food_categories

'Fruits, vegetables, whole grains, lean proteins, dairy or dairy alternatives, healthy fats, and legumes.'

In [182]:
Food_prompt_1 = f"""
Please add to the list seeds and nuts, and any other categories that might be missing. 
Please be comprehensive. 
"""

In [183]:
Food_categories = chat_with_context(Food_prompt_1)

In [184]:
Food_categories

'Fruits, vegetables, whole grains, lean proteins, dairy or dairy alternatives, healthy fats, legumes, seeds, nuts, and herbs/spices.'

In [185]:
Food_prompt_2 = f"""
What specific factors about food would you consider in order to assess its nutritional value as part of a diet? \
Please, include as factors whether the food is gluten free, glycemic index and trans fats content.
Your output should be only the list of relevant factors, nothing else.
"""

### Determining important food factors

In [186]:
Food_factors = chat_with_context(Food_prompt_2)

In [187]:
Food_factors

'Nutritional composition, gluten-free status, glycemic index, and trans fats content.'

In [188]:
Food_prompt_3 = f"""
Please add to the relevant factor list other aspects that might be relevant for the daily nutritional needs \
of a 46 years old male. Please be comprehensive. \
Your output should be only the new list of relevant factors, nothing else.
"""

In [189]:
Food_factors = chat_with_context(Food_prompt_3)
Food_factors

'Nutritional composition, gluten-free status, glycemic index, trans fats content, calorie content, macronutrient distribution (carbohydrates, proteins, fats), fiber content, vitamins and minerals, sodium content, cholesterol content, and allergen information.'

### Building a JSON with all ingredients

In [190]:
Food_prompt_4 = f"""
Using our food categories list and relevant factor list, please \
generate a JSON data structure containing a comprehensive collection of food ingredients. \
For each ingredient include the relevant factor information. \
Your output should be only a well-formatted JSON including all the ingredients you know.\
The structure of the JSON data structure must have two levels: 1) food category and 2) ingredient's name.
"""

In [191]:
Ingredients = chat_with_context(Food_prompt_4)

In [206]:
print(Ingredients)

{
  "Fruits": {
    "Apple": {
      "Nutritional Composition": "Low in calories, high in fiber and vitamin C",
      "Gluten-Free": true,
      "Glycemic Index": 39,
      "Trans Fats Content": 0
    },
    "Banana": {
      "Nutritional Composition": "Good source of potassium, vitamin C, and dietary fiber",
      "Gluten-Free": true,
      "Glycemic Index": 51,
      "Trans Fats Content": 0
    },
    ...
  },
  "Vegetables": {
    "Broccoli": {
      "Nutritional Composition": "Rich in fiber, vitamin C, and folate",
      "Gluten-Free": true,
      "Glycemic Index": 10,
      "Trans Fats Content": 0
    },
    "Carrot": {
      "Nutritional Composition": "High in vitamin A, fiber, and antioxidants",
      "Gluten-Free": true,
      "Glycemic Index": 39,
      "Trans Fats Content": 0
    },
    ...
  },
  "Whole Grains": {
    "Quinoa": {
      "Nutritional Composition": "High in protein, fiber, and essential amino acids",
      "Gluten-Free": true,
      "Glycemic Index": 53,
      

In [ ]:
# ChatGPT gave us a example with elipsis, which won't parse well. 
# Remove them before parsing
Ingredients_fixed = Ingredients.replace(",\n    ...\n","")

In [209]:
ingredients_json = json.loads(Ingredients_fixed)

In [210]:
# Food categories
print(list(ingredients_json.keys()))

['Fruits', 'Vegetables', 'Whole Grains', 'Lean Proteins', 'Dairy or Dairy Alternatives', 'Healthy Fats', 'Legumes', 'Seeds', 'Nuts', 'Herbs/Spices']


In [211]:
# Lists of ingredients
for category in ingredients_json.keys():
    print(category +": " +str(list(ingredients_json.get(category).keys())))

Fruits: ['Apple', 'Banana']
Vegetables: ['Broccoli', 'Carrot']
Whole Grains: ['Quinoa', 'Brown Rice']
Lean Proteins: ['Chicken Breast', 'Salmon']
Dairy or Dairy Alternatives: ['Greek Yogurt', 'Almond Milk']
Healthy Fats: ['Avocado', 'Olive Oil']
Legumes: ['Chickpeas', 'Lentils']
Seeds: ['Chia Seeds', 'Flaxseeds']
Nuts: ['Almonds', 'Walnuts']
Herbs/Spices: ['Basil', 'Turmeric']


In [212]:
# Details of ingredient
ingredients_json.get('Lean Proteins').get('Chicken Breast')

{'Nutritional Composition': 'High in protein, low in fat',
 'Gluten-Free': True,
 'Glycemic Index': 0,
 'Trans Fats Content': 0}

In [213]:
# Details of ingredient
ingredients_json.get('Vegetables').get('Broccoli')

{'Nutritional Composition': 'Rich in fiber, vitamin C, and folate',
 'Gluten-Free': True,
 'Glycemic Index': 10,
 'Trans Fats Content': 0}

In [221]:
# Are we close to max tokens (4k)??
num_tokens_from_messages(context) 

1668

In [224]:
Food_prompt_5 = f"""
Add the following ingredients to the JSON (including their relevant factors): 
###
Fruits: ['pineapple', 'strawberry', 'grapes', 'pears', 'mango', 'kiwi', 'orange', 'peach', 'plum', 'apricot']
Vegetables: ['mushrooms', 'artichoke', 'aubergine', 'asparagus', 'caouliflower', 'green beans', 'chard', 'zucchini']
Whole Grains: ['oatmeal', 'whole wheat']
Lean Proteins: ['lean beef', 'White-fleshed fish']
Dairy or Dairy Alternatives: ['kefir']
Legumes: ['soybeans', 'peanuts', 'green peas']
Seeds: ['sunflower seeds', 'pumpkim pipes']
###
Your output should be only the new JSON with no elipsis. 
"""

In [227]:
Ingredients = chat_with_context(Food_prompt_5)

In [231]:
print(Ingredients)

{
  "Fruits": {
    "Apple": {
      "Nutritional Composition": "Low in calories, high in fiber and vitamin C",
      "Gluten-Free": true,
      "Glycemic Index": 39,
      "Trans Fats Content": 0
    },
    "Banana": {
      "Nutritional Composition": "Good source of potassium, vitamin C, and dietary fiber",
      "Gluten-Free": true,
      "Glycemic Index": 51,
      "Trans Fats Content": 0
    },
    "Pineapple": {
      "Nutritional Composition": "Rich in vitamin C, manganese, and bromelain",
      "Gluten-Free": true,
      "Glycemic Index": 59,
      "Trans Fats Content": 0
    },
    "Strawberry": {
      "Nutritional Composition": "High in vitamin C, fiber, and antioxidants",
      "Gluten-Free": true,
      "Glycemic Index": 40,
      "Trans Fats Content": 0
    },
    "Grapes": {
      "Nutritional Composition": "Good source of antioxidants and vitamins",
      "Gluten-Free": true,
      "Glycemic Index": 46,
      "Trans Fats Content": 0
    },
    "Pears": {
      "Nutritio

In [232]:
# And with this we reached the token limit: 
num_tokens_from_messages(context) 

4101

In [265]:
# Anyhow, let's fix what we have so far in the JSON
Ingredients = """
{
  "Fruits": {
    "Apple": {
      "Nutritional Composition": "Low in calories, high in fiber and vitamin C",
      "Gluten-Free": true,
      "Glycemic Index": 39,
      "Trans Fats Content": 0
    },
    "Banana": {
      "Nutritional Composition": "Good source of potassium, vitamin C, and dietary fiber",
      "Gluten-Free": true,
      "Glycemic Index": 51,
      "Trans Fats Content": 0
    },
    "Pineapple": {
      "Nutritional Composition": "Rich in vitamin C, manganese, and bromelain",
      "Gluten-Free": true,
      "Glycemic Index": 59,
      "Trans Fats Content": 0
    },
    "Strawberry": {
      "Nutritional Composition": "High in vitamin C, fiber, and antioxidants",
      "Gluten-Free": true,
      "Glycemic Index": 40,
      "Trans Fats Content": 0
    },
    "Grapes": {
      "Nutritional Composition": "Good source of antioxidants and vitamins",
      "Gluten-Free": true,
      "Glycemic Index": 46,
      "Trans Fats Content": 0
    },
    "Pears": {
      "Nutritional Composition": "High in fiber, vitamin C, and antioxidants",
      "Gluten-Free": true,
      "Glycemic Index": 38,
      "Trans Fats Content": 0
    },
    "Mango": {
      "Nutritional Composition": "Rich in vitamin C, vitamin A, and fiber",
      "Gluten-Free": true,
      "Glycemic Index": 51,
      "Trans Fats Content": 0
    },
    "Kiwi": {
      "Nutritional Composition": "High in vitamin C, vitamin K, and fiber",
      "Gluten-Free": true,
      "Glycemic Index": 53,
      "Trans Fats Content": 0
    },
    "Orange": {
      "Nutritional Composition": "Rich in vitamin C, folate, and antioxidants",
      "Gluten-Free": true,
      "Glycemic Index": 43,
      "Trans Fats Content": 0
    },
    "Peach": {
      "Nutritional Composition": "Good source of vitamin C, vitamin A, and fiber",
      "Gluten-Free": true,
      "Glycemic Index": 42,
      "Trans Fats Content": 0
    },
    "Plum": {
      "Nutritional Composition": "High in vitamin C, vitamin K, and fiber",
      "Gluten-Free": true,
      "Glycemic Index": 39,
      "Trans Fats Content": 0
    },
    "Apricot": {
      "Nutritional Composition": "Rich in vitamin A, vitamin C, and fiber",
      "Gluten-Free": true,
      "Glycemic Index": 34,
      "Trans Fats Content": 0
    }
  },
  "Vegetables": {
    "Broccoli": {
      "Nutritional Composition": "Rich in fiber, vitamin C, and folate",
      "Gluten-Free": true,
      "Glycemic Index": 10,
      "Trans Fats Content": 0
    },
    "Carrot": {
      "Nutritional Composition": "High in vitamin A, fiber, and antioxidants",
      "Gluten-Free": true,
      "Glycemic Index": 39,
      "Trans Fats Content": 0
    },
    "Mushrooms": {
      "Nutritional Composition": "Low in calories, high in antioxidants and vitamins",
      "Gluten-Free": true,
      "Glycemic Index": 10,
      "Trans Fats Content": 0
    },
    "Artichoke": {
      "Nutritional Composition": "Good source of fiber, vitamin C, and antioxidants",
      "Gluten-Free": true,
      "Glycemic Index": 15,
      "Trans Fats Content": 0
    },
    "Aubergine": {
      "Nutritional Composition": "Low in calories, high in fiber and antioxidants",
      "Gluten-Free": true,
      "Glycemic Index": 15,
      "Trans Fats Content": 0
    },
    "Asparagus": {
      "Nutritional Composition": "Low in calories, high in fiber and vitamins",
      "Gluten-Free": true,
      "Glycemic Index": 15,
      "Trans Fats Content": 0
    },
    "Cauliflower": {
      "Nutritional Composition": "Low in calories, high in fiber and vitamins",
      "Gluten-Free": true,
      "Glycemic Index": 15,
      "Trans Fats Content": 0
    },
    "Green Beans": {
      "Nutritional Composition": "Low in calories, high in fiber and vitamins",
      "Gluten-Free": true,
      "Glycemic Index": 15,
      "Trans Fats Content": 0
    },
    "Chard": {
      "Nutritional Composition": "Rich in vitamins, minerals, and antioxidants",
      "Gluten-Free": true,
      "Glycemic Index": 15,
      "Trans Fats Content": 0
    },
    "Zucchini": {
      "Nutritional Composition": "Low in calories, high in fiber and vitamins",
      "Gluten-Free": true,
      "Glycemic Index": 15,
      "Trans Fats Content": 0
    }
  },
  "Whole Grains": {
    "Quinoa": {
      "Nutritional Composition": "High in protein, fiber, and essential amino acids",
      "Gluten-Free": true,
      "Glycemic Index": 53,
      "Trans Fats Content": 0
    },
    "Brown Rice": {
      "Nutritional Composition": "Good source of fiber and minerals",
      "Gluten-Free": true,
      "Glycemic Index": 50,
      "Trans Fats Content": 0
    },
    "Oatmeal": {
      "Nutritional Composition": "High in fiber, vitamins, and minerals",
      "Gluten-Free": true,
      "Glycemic Index": 55,
      "Trans Fats Content": 0
    },
    "Whole Wheat": {
      "Nutritional Composition": "Good source of fiber and nutrients",
      "Gluten-Free": false,
      "Glycemic Index": 69,
      "Trans Fats Content": 0
    }
  },
  "Lean Proteins": {
    "Chicken Breast": {
      "Nutritional Composition": "High in protein, low in fat",
      "Gluten-Free": true,
      "Glycemic Index": 0,
      "Trans Fats Content": 0
    },
    "Salmon": {
      "Nutritional Composition": "Rich in omega-3 fatty acids and high-quality protein",
      "Gluten-Free": true,
      "Glycemic Index": 0,
      "Trans Fats Content": 0
    },
    "Lean Beef": {
      "Nutritional Composition": "Good source of protein, iron, and vitamins",
      "Gluten-Free": true,
      "Glycemic Index": 0,
      "Trans Fats Content": 0
    },
    "White-fleshed Fish": {
      "Nutritional Composition": "Low in fat, high in protein and omega-3 fatty acids",
      "Gluten-Free": true,
      "Glycemic Index": 0,
      "Trans Fats Content": 0
    }
  },
  "Dairy or Dairy Alternatives": {
    "Greek Yogurt": {
      "Nutritional Composition": "Good source of protein and calcium",
      "Gluten-Free": true,
      "Glycemic Index": 11,
      "Trans Fats Content": 0
    },
    "Almond Milk": {
      "Nutritional Composition": "Low in calories, no cholesterol, and lactose-free",
      "Gluten-Free": true,
      "Glycemic Index": 25,
      "Trans Fats Content": 0
    },
    "Kefir": {
      "Nutritional Composition": "Rich in probiotics, calcium, and vitamins",
      "Gluten-Free": true,
      "Glycemic Index": 30,
      "Trans Fats Content": 0
    }
  },
  "Legumes": {
    "Chickpeas": {
      "Nutritional Composition": "Good source of protein, fiber, and minerals",
      "Gluten-Free": true,
      "Glycemic Index": 28,
      "Trans Fats Content": 0
    },
    "Lentils": {
      "Nutritional Composition": "High in protein, fiber, and folate",
      "Gluten-Free": true,
      "Glycemic Index": 32,
      "Trans Fats Content": 0
    },
    "Soybeans": {
      "Nutritional Composition": "Rich in protein, fiber, and minerals",
      "Gluten-Free": true,
      "Glycemic Index": 15,
      "Trans Fats Content": 0
    },
    "Peanuts": {
      "Nutritional Composition": "Good source of protein, healthy fats, and vitamins",
      "Gluten-Free": true,
	  "Glycemic Index": 14,
      "Trans Fats Content": 0
	}
  },
  "Seeds": {
    "Chia Seeds": {
      "Nutritional Composition": "Rich in omega-3 fatty acids, fiber, and antioxidants",
      "Gluten-Free": true,
      "Glycemic Index": 1,
      "Trans Fats Content": 0
    },
    "Flaxseeds": {
      "Nutritional Composition": "High in omega-3 fatty acids, fiber, and lignans",
      "Gluten-Free": true,
      "Glycemic Index": 0,
      "Trans Fats Content": 0
    }
  },
  "Nuts": {
    "Almonds": {
      "Nutritional Composition": "Good source of healthy fats, protein, and fiber",
      "Gluten-Free": true,
      "Glycemic Index": 0,
      "Trans Fats Content": 0
    },
    "Walnuts": {
      "Nutritional Composition": "Rich in omega-3 fatty acids and antioxidants",
      "Gluten-Free": true,
      "Glycemic Index": 0,
      "Trans Fats Content": 0
    }
  },
  "Herbs/Spices": {
    "Basil": {
      "Nutritional Composition": "Rich in antioxidants and essential oils",
      "Gluten-Free": true,
      "Glycemic Index": 0,
      "Trans Fats Content": 0
    },
    "Turmeric": {
      "Nutritional Composition": "Anti-inflammatory properties and rich in antioxidants",
      "Gluten-Free": true,
      "Glycemic Index": 0,
      "Trans Fats Content": 0
    }
  }
}
"""
ingredients_json_2 = json.loads(Ingredients)

In [230]:
num_tokens_from_messages(context) 

4101

In [239]:
# Lists of ingredients
for category in ingredients_json_2.keys():
    print(category +": " +str(list(ingredients_json_2.get(category).keys())))

Fruits: ['Apple', 'Banana', 'Pineapple', 'Strawberry', 'Grapes', 'Pears', 'Mango', 'Kiwi', 'Orange', 'Peach', 'Plum', 'Apricot']
Vegetables: ['Broccoli', 'Carrot', 'Mushrooms', 'Artichoke', 'Aubergine', 'Asparagus', 'Cauliflower', 'Green Beans', 'Chard', 'Zucchini']
Whole Grains: ['Quinoa', 'Brown Rice', 'Oatmeal', 'Whole Wheat']
Lean Proteins: ['Chicken Breast', 'Salmon', 'Lean Beef', 'White-fleshed Fish']
Dairy or Dairy Alternatives: ['Greek Yogurt', 'Almond Milk', 'Kefir']
Legumes: ['Chickpeas', 'Lentils', 'Soybeans', 'Peanuts']
Seeds: ['Chia Seeds', 'Flaxseeds']
Nuts: ['Almonds', 'Walnuts']
Herbs/Spices: ['Basil', 'Turmeric']


In [244]:
# Details of ingredient
ingredients_json_2.get('Lean Proteins').get('White-fleshed Fish')

{'Nutritional Composition': 'Low in fat, high in protein and omega-3 fatty acids',
 'Gluten-Free': True,
 'Glycemic Index': 0,
 'Trans Fats Content': 0}

In [215]:
# End of 1st conversation
context

[{'role': 'system',
  'content': '\nYou are a nutrition expert agent with predilection for the mediterranean diet and extensive knowledge about healthy nutritional habits. You are also an expert in gluten-free diets.\nYour mission is to answer questions about nutrition and cooking, focusing on the reducing the risk of metabolic syndrome.\n'},
 {'role': 'user',
  'content': '\nWhat are all the food categories to be considered when designing a complete diet for a male adult?\nYour output should be only the list of food categories, nothing else. \n'},
 {'role': 'assistant',
  'content': 'Fruits, vegetables, whole grains, lean proteins, dairy or dairy alternatives, healthy fats, and legumes.'},
 {'role': 'user',
  'content': '\nPlease add to the list seeds and nuts, and any other categories that might be missing. \nPlease be comprehensive. \n'},
 {'role': 'assistant',
  'content': 'Fruits, vegetables, whole grains, lean proteins, dairy or dairy alternatives, healthy fats, legumes, seeds, n

## Second conversation

In [258]:
# Starting with an empty context
context = []
# Same role
context.append({'role':'system', 'content':system_role_prompt})

### Generating a diet plan

In [261]:
Diet_prompt_0 = f"""
Please, write a diet for a complete week, including 5 meals per day.
For each meal specify a recipe using only gluten free ingredients from the list below:
####
{Ingredients}
####
"""

In [269]:
# Can't use this prompt as it exceeds the max tokens
# num_tokens_from_messages(context)  

In [272]:
# Getting just a list of gluten free ingredients
gf_list = []
for category in ingredients_json_2.keys():
    for item in ingredients_json_2.get(category).keys():
        if ingredients_json_2.get(category).get(item).get('Gluten-Free') == True: 
            gf_list.append(item)
print(gf_list)

['Apple', 'Banana', 'Pineapple', 'Strawberry', 'Grapes', 'Pears', 'Mango', 'Kiwi', 'Orange', 'Peach', 'Plum', 'Apricot', 'Broccoli', 'Carrot', 'Mushrooms', 'Artichoke', 'Aubergine', 'Asparagus', 'Cauliflower', 'Green Beans', 'Chard', 'Zucchini', 'Quinoa', 'Brown Rice', 'Oatmeal', 'Chicken Breast', 'Salmon', 'Lean Beef', 'White-fleshed Fish', 'Greek Yogurt', 'Almond Milk', 'Kefir', 'Chickpeas', 'Lentils', 'Soybeans', 'Peanuts', 'Chia Seeds', 'Flaxseeds', 'Almonds', 'Walnuts', 'Basil', 'Turmeric']


In [273]:
Diet_prompt_1 = f"""
Please, write a diet for a complete week, including 5 meals per day.
For each meal specify a recipe using only ingredients from the list below:
####
{gf_list}
####
"""

In [275]:
diet = chat_with_context(Diet_prompt_1)

In [277]:
print(diet)

Sure! Here's a sample diet plan for a week, including 5 meals per day, using the ingredients from the list provided:

Day 1:
Breakfast: Oatmeal with sliced banana and a sprinkle of chia seeds.
Snack: Greek yogurt with mixed berries (strawberries, blueberries, raspberries).
Lunch: Grilled chicken breast with a side of steamed broccoli and brown rice.
Snack: Carrot sticks with hummus.
Dinner: Baked salmon with roasted asparagus and quinoa.
 
Day 2:
Breakfast: Scrambled eggs with sautéed mushrooms and spinach.
Snack: Apple slices with almond butter.
Lunch: Lentil salad with mixed greens, cherry tomatoes, and a lemon-turmeric dressing.
Snack: Greek yogurt with sliced peaches and a sprinkle of flaxseeds.
Dinner: Grilled lean beef with roasted zucchini and a side of quinoa.

Day 3:
Breakfast: Smoothie made with almond milk, mixed berries, and a handful of spinach.
Snack: Greek yogurt with sliced mango and a sprinkle of almonds.
Lunch: Grilled chicken breast with a side of roasted cauliflower

In [279]:
Diet_prompt_2 = f"""
Could you please translate the diet plan to Spanish (Spain)"""

In [280]:
diet_es = chat_with_context(Diet_prompt_2)

In [281]:
print(diet_es)

¡Claro! Aquí tienes un plan de dieta de muestra para una semana, que incluye 5 comidas al día, utilizando los ingredientes de la lista proporcionada:

Día 1:
Desayuno: Avena con plátano en rodajas y una pizca de semillas de chía.
Merienda: Yogur griego con bayas mixtas (fresas, arándanos, frambuesas).
Almuerzo: Pechuga de pollo a la parrilla con un lado de brócoli al vapor y arroz integral.
Merienda: Bastones de zanahoria con hummus.
Cena: Salmón al horno con espárragos asados y quinoa.

Día 2:
Desayuno: Huevos revueltos con champiñones salteados y espinacas.
Merienda: Rodajas de manzana con mantequilla de almendras.
Almuerzo: Ensalada de lentejas con mezcla de lechugas, tomates cherry y aderezo de limón y cúrcuma.
Merienda: Yogur griego con melocotones en rodajas y una pizca de semillas de lino.
Cena: Carne magra a la parrilla con calabacín asado y un lado de quinoa.

Día 3:
Desayuno: Batido hecho con leche de almendras, bayas mixtas y un puñado de espinacas.
Merienda: Yogur griego co